In [1]:
!pwd


/Users/rayduong/Documents/GitHub/scrgt-bdso-cc-datascience/genreClassification


In [29]:
import os
# os.chdir('/home/jovyan/scrgt-bdso-cc-datascience/datapipeline'
os.chdir('/Users/rayduong/Documents/GitHub/scrgt-bdso-cc-datascience/datapipeline')

os.environ["ESHOST"] ='localhost'
os.environ["ESPORT"] ='9200'
ESHOST= os.environ.get('ESHOST')
ESPORT= os.environ.get('ESPORT')



# Extract list of movies by actor by genre

In [30]:
import os
import sys
import time
import logging
import numpy as np
import pandas as pd
from datetime import datetime
import urllib.parse
import http.client
import json
import logging
logger = logging.getLogger('personCasting')
logger.setLevel(logging.INFO)

In [50]:
import numpy as np
import nltk
from nltk.stem import PorterStemmer

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopwords=  set(stopwords.words('english'))

from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rayduong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rayduong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
path_to_json = './data/cast/themoviedb/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') ]
len(json_files)

theMovieDB=[]
for fname in json_files:
    try:
        with open(fname, 'r') as f:
            r = json.load(f)   
            for row in r:
                theMovieDB.append(row)

    except Exception as e:
        #extract.update({ columns[key]  : '' } )
        logger.info('Unable to find key : '+ fname + '::' +str(e))        
        

In [32]:
json_files[:1]

['./data/cast/themoviedb/themoviedb_c_21.json']

In [33]:
theMovieDB=[]
for fname in json_files:
    try:
        with open(fname, 'r') as f:
            r = json.load(f)   
            for row in r:
                theMovieDB.append(row)

    except Exception as e:
        #extract.update({ columns[key]  : '' } )
        logger.info('Unable to find key : '+ fname + '::' +str(e))        
        
        

In [34]:
# theMovieDB=[]
len(theMovieDB)


2741

In [48]:
castDF =  pd.DataFrame.from_dict(theMovieDB , dtype=str)
castDF.head()

,_actorID,_source,actor,actorID,character,characterID,created_utc_dt,movieID,movieTitle
0,21,themoviedb,Adam Sandler,19292,George Simmons,52fe43fcc3a368484e0092f7,"09/09/2019, 18:02:25",20829,Funny People
1,21,themoviedb,Adam Sandler,19292,Monkey (voice),52fe46aa9251416c9105bb25,"09/09/2019, 18:02:25",38317,Zookeeper
2,21,themoviedb,Adam Sandler,19292,Lenny Feder,52fe46b29251416c9105c999,"09/09/2019, 18:02:25",38365,Grown Ups
3,21,themoviedb,Adam Sandler,19292,Danny Maccabee,52fe47cbc3a36847f81488b1,"09/09/2019, 18:02:25",50546,Just Go with It
4,21,themoviedb,Adam Sandler,19292,Jack / Jill,52fe484dc3a368484e0f2067,"09/09/2019, 18:02:25",71880,Jack and Jill


In [36]:
path_to_json = './data/movies/wikipedia/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and os.stat(path_to_json+pos_json).st_size>0]
len(json_files)

moviesPlot=[]

for fname in json_files:
    with open(fname, 'r') as f:
        r = json.load(f)   
        columns= {'_movieID': '_movieID', 'plot': 'plot', 'summary': 'summary' }
        
        extract ={}
        for key in columns:
            try: 
                extract.update({ columns[key] : r[key] } )
            except Exception as e:
                extract.update({ columns[key]  : '' } )
                logger.info('Unable to find key : '+ key + '::' +str(e))

        moviesPlot.append(extract)
    
    

In [37]:
moviesPlot[1:10]


[{'_movieID': '392217', 'plot': '', 'summary': ''},
 {'_movieID': '578547', 'plot': '', 'summary': ''},
 {'_movieID': '341689',
  'plot': '',
  'summary': 'For the film based on the story, see How to Talk to Girls at Parties (film)"How to Talk to Girls at Parties" is a science fiction short story written in 2006 by Neil Gaiman.\nIt is about a couple of British 1970s teen-aged boys, Enn and Vic, who go to a party to meet girls, only to find that the girls are very different from the boys\' expectations. \n"How to Talk to Girls at Parties" was nominated for the 2007 Hugo Award for Best Short Story and won the Locus Award for Best Short Story.  In 2009 it was the inspiration for a limited-edition poster with art by Camilla d\'Errico, which was sold through Gaiman\'s merchandising site Neverwear. The poster had a limited run of 1200 numbered copies, and 10% of the proceeds were donated to the Comic Book Legal Defense Fund, a favourite charity of Gaiman\'s.'},
 {'_movieID': '20178',
  'plot

In [38]:
plotDF =  pd.DataFrame.from_dict(moviesPlot , dtype=str)
plotDF[plotDF['plot']==''].head()

,_movieID,plot,summary
0,574837,,
1,392217,,
2,578547,,
3,341689,,"For the film based on the story, see How to Ta..."
6,50780,,"The beaver (genus Castor) is a large, primaril..."


In [39]:
path_to_json = './data/movies/imdb/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and os.stat(path_to_json+pos_json).st_size>0]
len(json_files)

movieGenre =[]

for fname in json_files:
    with open(fname, 'r') as f:
        r = json.load(f)   
        columns= {'_id': '_movieID', 'plot outline': 'plot', 'genres': 'genres' }
        
        extract ={}
        for key in columns:
            try: 
                if key == 'genres':
                    extract.update({'genres': ','.join(r[key]) })
                else:
                    extract.update({ columns[key] : r[key] } )
            except Exception as e:
                extract.update({ columns[key]  : '' } )
                logger.info('Unable to find key : '+ key + '::' +str(e))

        movieGenre.append(extract)
    
    

In [13]:
movieGenreDF =  pd.DataFrame.from_dict(movieGenre , dtype=str)
movieGenreDF.head()

,_movieID,genres,plot
0,376530,"Documentary,Adventure,Sport",This spellbinding documentary follows Aisholpa...
1,38319,"Comedy,Fantasy","Throughout history, tales of chivalry have bur..."
2,312221,"Drama,Sport",Adonis Johnson is the son of the famous boxing...
3,178619,"Documentary,Short",
4,354857,"Animation,Action,Adventure,Comedy,Family,Fanta...","After a future, dying Rigby comes to warn the ..."


In [40]:
plotDF.head()

,_movieID,plot,summary
0,574837,,
1,392217,,
2,578547,,
3,341689,,"For the film based on the story, see How to Ta..."
4,20178,Lance Clayton (Robin Williams) is a single fa...,World's Greatest Dad is a 2009 American satiri...


In [41]:
castDF.head()

,_actorID,_source,actor,actorID,character,characterID,created_utc_dt,movieID,movieTitle
0,21,themoviedb,Adam Sandler,19292,George Simmons,52fe43fcc3a368484e0092f7,"09/09/2019, 18:02:25",20829,Funny People
1,21,themoviedb,Adam Sandler,19292,Monkey (voice),52fe46aa9251416c9105bb25,"09/09/2019, 18:02:25",38317,Zookeeper
2,21,themoviedb,Adam Sandler,19292,Lenny Feder,52fe46b29251416c9105c999,"09/09/2019, 18:02:25",38365,Grown Ups
3,21,themoviedb,Adam Sandler,19292,Danny Maccabee,52fe47cbc3a36847f81488b1,"09/09/2019, 18:02:25",50546,Just Go with It
4,21,themoviedb,Adam Sandler,19292,Jack / Jill,52fe484dc3a368484e0f2067,"09/09/2019, 18:02:25",71880,Jack and Jill


In [42]:
castDF['movieID'] = castDF['movieID'].astype(str)
castMoivesDF = castDF.merge( plotDF, how='inner', left_on = 'movieID', right_on = '_movieID')
castMoivesDF = castMoivesDF.merge( movieGenreDF, how='inner', left_on = '_movieID', right_on = '_movieID')

castMoivesDF['plotCombine'] = castMoivesDF['movieTitle']+ ' '+ castMoivesDF['plot_x']+' '+castMoivesDF['summary']+' '+castMoivesDF['plot_y']
extractDF=castMoivesDF[['_actorID', 'actor', '_movieID', 'plotCombine', 'genres']]
extractDF.head()

,_actorID,actor,_movieID,plotCombine,genres
0,21,Adam Sandler,20829,Funny People George Simmons is a middle-aged r...,"Comedy,Drama"
1,21,Adam Sandler,38365,"Grown Ups In 1978, five 12-year-olds win a C...",Comedy
2,21,Adam Sandler,76492,"Hotel Transylvania In 1895, in the aftermath o...","Animation,Comedy,Family,Fantasy,Horror"
3,21,Adam Sandler,87428,That's My Boy The film begins in Massachusetts...,Comedy
4,21,Adam Sandler,232672,Blended After a blind date gone horribly wro...,"Comedy,Romance"


In [43]:
# Combine all the movie plots from different data sources into one field
# Given a moive can belong into multiple genre, create row for each movie plot

extractDF = extractDF['genres'].str.split(',').apply(pd.Series) \
.merge(extractDF, right_index = True, left_index=True) \
.drop(['genres'], axis =1) \
.melt(id_vars =['_actorID', 'actor', '_movieID', 'plotCombine'], value_name='genre') \
.drop('variable', axis=1) \
.dropna()


In [44]:
Test 
extractDF[extractDF['_movieID']=='76492']

,_actorID,actor,_movieID,plotCombine,genre
2,21,Adam Sandler,76492,"Hotel Transylvania In 1895, in the aftermath o...",Animation
1307,21,Adam Sandler,76492,"Hotel Transylvania In 1895, in the aftermath o...",Comedy
2612,21,Adam Sandler,76492,"Hotel Transylvania In 1895, in the aftermath o...",Family
3917,21,Adam Sandler,76492,"Hotel Transylvania In 1895, in the aftermath o...",Fantasy
5222,21,Adam Sandler,76492,"Hotel Transylvania In 1895, in the aftermath o...",Horror


In [45]:
# Combine all the different movie plots by actor and genre.

actorGenreSummary = extractDF.groupby(['_actorID', 'actor', 'genre'])['plotCombine'].apply(lambda x: ' '.join(x)).reset_index()
actorGenreSummary.head()

,_actorID,actor,genre,plotCombine
0,1,Robert Downey Jr.,Action,Sherlock Holmes Sherlock Holmes ( or ) is a f...
1,1,Robert Downey Jr.,Adventure,"Due Date Peter Highman (Robert Downey Jr.), a ..."
2,1,Robert Downey Jr.,Biography,"I Am Steve McQueen I Am Steve McQueen"" is to..."
3,1,Robert Downey Jr.,Comedy,"Due Date Peter Highman (Robert Downey Jr.), a ..."
4,1,Robert Downey Jr.,Crime,The Judge A judge is a person who presides ov...


In [22]:
# Get counts by genre
extractDF.groupby('genre').count()

,_actorID,actor,_movieID,plotCombine
genre,,,,
,8,8,8,8
Action,269,269,269,269
Adventure,174,174,174,174
Animation,91,91,91,91
Biography,128,128,128,128
Comedy,398,398,398,398
Crime,173,173,173,173
Documentary,282,282,282,282
Drama,535,535,535,535


In [23]:
# Get a list of generes

', '.join(extractDF['genre'].unique().tolist())

'Comedy, Animation, Action, Documentary, Short, Adventure, Drama, Biography, Music, Thriller, Horror, Mystery, Crime, Sport, Musical, , Family, Game-Show, Reality-TV, Fantasy, Romance, Sci-Fi, History, War, Western, News, Film-Noir'

In [53]:
actorGenreCatalog = actorGenreSummary.to_json(orient='records')
actorGenreJson = eval(actorGenreCatalog)
len(actorGenreJson)

1228

In [54]:
# Save model lookup 
pickle.dump( actorGenreJson, open( "./models/actorGenreDict.pkl", "wb" ) )

In [55]:
sentencesAll = actorGenreSummary['plotCombine'].tolist()
len(sentencesAll) 

1228

In [56]:
def tokenizeDoc(docs, stopwords=stopwords ):
    i=0
    for doc in docs:
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(doc.lower())
        # Stem words
        #ps = PorterStemmer()
        #tokens = [ps.stem(word) for word in tokens]
        # Remove stop words.
        tokens = [word for word in tokens if word not in stopwords]
        fdist1 = nltk.FreqDist(tokens)
        words = list(word for word, freq in fdist1.items() if not word.isdigit())

        docs[i] = ' '.join(words[:3000])
        i += 1
    return docs

In [57]:
def cleanSentences(sentences):
    punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'  #TBD
        #TBD - find sentence splitter - 
        # remove punctuation, strip white space; remove non-standard characters; remove numbers; 
    i = 0
    for s in sentences:
        s = s.strip()
        s = s.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').replace("[0-9]", " ")
        s = s.replace(',', '').replace('.','') #TBD - replace with punctuation
        #s = s[0:2000]
        sentences[i] = s
        i += 1
    return sentences

In [58]:
sentences = tokenizeDoc(sentencesAll)

In [59]:
sentences[1]

'due date peter highman robert downey jr successful architect fly home atlanta los angeles wife sarah michelle monaghan give birth way airport chance encounter ethan tremblay zach galifianakis going la actor planning scatter recently deceased father ashes grand canyon misuses words terrorist bomb talking escorted plane first series misadventures caused drastically dysfunctional list missing wallet agrees drive stops buy marijuana discovers nearly money since gets wire wired stage name instead legal western union employee danny mcbride refuses accept leads violent altercation night rest stop decides abandon realizes still forgotten unload stuff left causes wrestle conscience return drives get falls asleep wheel crashes car picked friend darryl jamie foxx part company initially persuades otherwise throws mistakenly drinking stored coffee tin lets use range rover make trip mexico united states border despite assuring handle situation flees arrested possession mexican federal police lock s

In [60]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

config = tf.ConfigProto()
config.intra_op_parallelism_threads = 200
config.gpu_options.allow_growth = True
tf.Session(config=config)


url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

elmodir = os.getcwd()+'/models/'

In [34]:
def embed_text(text):
    vectors = session.run(embeddings, feed_dict={text_ph: text})
    return [vector.tolist() for vector in vectors]

In [35]:
# score n Sentences at time - to pipeline this
def defineEmbeddings (start, end):    
    embeddings = embed(
        sentences[start:end],
        signature="default",
        as_dict=True)["default"]
    return embeddings# Save Weights - use names based on Movie 

import pickle as pkl
# score sentences
def scoreSentences (embeddings):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        x = sess.run(embeddings)
    return x

def saveWeights(elmoFN, x):
    with open('./models/'+elmoFN,'wb') as f: pkl.dump(x, f)
    return

In [ ]:
import math 

batch_size = 50                    # Specify number of sentences per file  (was 500)
numPlots = len(sentences)
numfiles = math.ceil(numPlots/batch_size) 

start = 0
end = batch_size
for file in range(1,numfiles+1):
    elmoFN = 'elmoPlotsweights' + str(start) + 'to' + str(end) + '.pkl'
    print('Score sentences:', elmoFN)

        # Define Embeddings, Score, and Store Weights one Batch of N at time
    embeddings = defineEmbeddings(start, end)
    x = scoreSentences(embeddings)
    saveWeights(elmoFN,x)
    start = end
    end = end+batch_size

In [ ]:
# Read Weights - use names based on Movie 
# TBD - add row num -- TBD Save with Movie ID 
# TBD - missing a row ERROR
import pickle as pkl
import numpy as np
import glob

def readElmoWeightFile(file):
    with open(elmodir+file, 'rb') as f:
        elmo_file = pkl.load(f)
    return list(elmo_file)
  
def readWeights(elmodir):
    noFiles = len([name for name in os.listdir(elmodir) ])
    xin = readElmoWeightFile('elmoPlotweights0to'+str(batch_size)+'.pkl')
    size = batch_size
    for x in range(1, noFiles):
        start = x*size
        end = start+size
        filename = 'elmoPlotweights' + str(start) + 'to' + str(end) + '.pkl'
        print(filename)
        xin += readElmoWeightFile(filename)
    return xin

In [ ]:
len(xin[0])

In [ ]:
#Export Dictonary and GenrePlot Vectors
import pickle
pickle.dump( actorGenreJson, open( "./models/actorGenreDict.pkl", "wb" ) )
pickle.dump( xin, open( "./models/actorGenreVect.pkl", "wb" ) )


# Push Model to Production

In [ ]:
actorGenreJson = pickle.load( open( '../models/actorGenreDict.pkl', 'rb' ) )
xin = pickle.load( open( '../models/actorGenreVect.pkl', 'rb' ) )


## Test Elasticsearch to store the vertor and peform cosine similarity score.


In [ ]:
import requests
from pprint import pprint
from datetime import datetime
from elasticsearch import Elasticsearch
import json


_index='genresplot'
esIndex = 'http://'+ESHOST+':'+ESPORT+'/'+_index  
es=Elasticsearch([{'host':ESHOST,'port':ESPORT}])

_doc= 'genreplot'

headers={"Content-Type": "application/json"}

In [ ]:
request_body = {
  "mappings": {
    "properties": {
      "_actorID" : {
        "type" : "keyword"
      },     
      "celebrity" : {
        "type" : "keyword"
      }, 
      "genre" : {
        "type" : "keyword"
      }, 
                
      "genre_vector": {
        "type": "dense_vector",
        "dims": 1024
      }

    }
  }
}

try:
    es.indices.delete(index = _index)
except:
    pass
    
es.indices.create(index = _index, body = request_body)

In [ ]:
actorGenreJson[0]

In [ ]:
# Load Facail data into ES
for i in range(len(actorGenreJson)):
    actorGenre =actorGenreJson[i]
    _id = actorGenre['_actorID']+'-'+actorGenre['genre']
    #print(_id)
    doc = {
        "_actorID" : actorGenre['_actorID'],
        "celebrity": actorGenre['actor'],
        "genre": actorGenre['genre'],
        "genre_vector": [float(x) for x in xin[i]]
        }
    r = requests.put(esIndex+'/_doc/'+_id, headers= headers, data = json.dumps(doc))
    print (r.text)

In [ ]:
def getCelbGenre( celebrity): 
    q={
      "_source": {
        "includes": [ "genre"]
        },
      "query": { 
        "bool": { 
          "must": [
            { "match": { "celebrity":celebrity}}
          ]
        }
      }
    }
    res= es.search(index=_index, body=json.dumps(q))
    if res['hits']['hits']:
        results=[]
        matches = res['hits']['hits']
        for match in matches:
            results.append(match['_source']['genre'])
        return results
    else:
        return None
            


In [ ]:
def getCelbGenreVec( celebrity, genre): 
    q={
      "query": { 
        "bool": { 
          "must": [
            { "match": { "celebrity":celebrity}}
          ],
          "filter": [ 
            { "term":  { "genre": genre }}
          ]
        }
      }
    }
    res= es.search(index=_index, body=json.dumps(q))
    if res['hits']['hits']:
        match = res['hits']['hits'][0]
        #print(match)
        return match['_source']['genre_vector']
    else:
        return None
        



In [ ]:
def getSimilarGenreVec( genre_vec, genre): 
    q={
          "query": {
            "script_score": {
             "query": {
                    "terms" : { "genre" : [genre]}
                },
              "script": {
                "source": "cosineSimilarity(params.query_vector, doc['genre_vector']) + 1.0", 
                "params": {
                  "query_vector": genre_vec
                }
              }
            }
          }
        }
    #print(json.dumps(q))
    res= es.search(index=_index, body=q)

    res= es.search(index=_index, body=json.dumps(q))
    if res['hits']['hits']:
        matches = res['hits']['hits']
        #print(match)
        results=[]
        for match in matches:
            results.append( { 'genre': genre, 'celebrity': match['_source']['celebrity'], 'score': match['_score']})
        return (results) 
    else:
        return None

    



In [ ]:
# 'Comedy, Animation, Action, Documentary, Short, Adventure, Drama, Biography, Music, Thriller, Horror, Mystery, Crime, Sport, Musical, , Family, Game-Show, Reality-TV, Fantasy, Romance, Sci-Fi, History, War, Western, News, Film-Noir'
celebritySearch = 'Robert Downey Jr.'
genreSearch='Adventure'

getCelbGenre(celebritySearch)


In [ ]:
genreVec = getCelbGenreVec(celebritySearch, genreSearch)
getSimilarGenreVec( genreVec, genreSearch)

# Recommendations Celebrity alterative based on their past genre movies.

* For a given Celebrity, reference all the genre we have information on. Retrieve the genre vector space based on 
the list of movies plots the actor performed in the past.
* Calculate the cosine distince and recommend other celebrities who performed in similar movies based on their plots 

In [ ]:
resByDrama=[]

for genreSearch in getCelbGenre(celebritySearch):
    genreVec = getCelbGenreVec(celebritySearch, genreSearch)
    genreRecs = getSimilarGenreVec( genreVec, genreSearch)
    # Remove the search celebrirty from the recomendations
    resByDrama = resByDrama+ [i  for i in genreRecs if i['celebrity']!= celebritySearch ] 
    
resByDrama

In [ ]:
from IPython.display import Image, HTML, display

display(HTML("<h2>Get similar celebrities for: %s</h2>"% celebritySearch))

pd.DataFrame.from_dict(resByDrama, dtype=str)   

In [ ]:

resultDF = pd.DataFrame.from_dict(resByDrama, dtype=str)   

resultDF.info()